In [5]:
import scipy.io as sio
import os 
import glob
import numpy as np

import pandas as pd

import matplotlib.pyplot as plt
import plotly.graph_objects as go
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import GradientBoostingRegressor

from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.preprocessing import RobustScaler,StandardScaler,MinMaxScaler,Normalizer,MaxAbsScaler
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.ensemble import VotingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

from xgboost import XGBRegressor 


from joblib import dump, load

from imutils import face_utils

import dlib
import cv2

import mediapipe

In [6]:
def draw_axis(img,pitch, yaw,  roll, tdx=None, tdy=None, size = 100):
    
    if tdx != None and tdy != None:
        tdx = tdx
        tdy = tdy
    else:
        height, width = img.shape[:2]
        tdx = width / 2
        tdy = height / 2

    # X-Axis pointing to right. drawn in red
    x1 = size * (np.cos(yaw) * np.cos(roll)) + tdx
    y1 = size * (np.cos(pitch) * np.sin(roll) + np.cos(roll) * np.sin(pitch) * np.sin(yaw)) + tdy

    # Y-Axis | drawn in green
    #        v
    x2 = size * (-np.cos(yaw) * np.sin(roll)) + tdx
    y2 = size * (np.cos(pitch) * np.cos(roll) - np.sin(pitch) * np.sin(yaw) * np.sin(roll)) + tdy

    # Z-Axis (out of the screen) drawn in blue
    x3 = size * (np.sin(yaw)) + tdx
    y3 = size * (-np.cos(yaw) * np.sin(pitch)) + tdy

    cv2.line(img, (int(tdx), int(tdy)), (int(x1),int(y1)),(0,0,255),3)
    cv2.line(img, (int(tdx), int(tdy)), (int(x2),int(y2)),(0,255,0),3)
    cv2.line(img, (int(tdx), int(tdy)), (int(x3),int(y3)),(255,0,0),2)

    return img

In [10]:
# df = pd.read_csv('imagesPointData__raduis__mediapipe.csv',)
df = pd.concat([pd.read_csv('yimagesPointData__raduis__mediapipe.csv',),pd.read_csv('yimagesPointData__raduis__mediapipe.csv',)],axis=0)
df

,0,1,2,3,4,5,6,7,8,9,...,930,931,932,933,934,935,pitch,yaw,roll,image
0,0.467596,0.466501,0.470845,0.461489,0.467716,0.471568,0.482683,0.370984,0.488630,0.490713,...,0.643605,0.556000,0.558392,0.558702,0.552916,0.545472,0.224714,0.078211,0.080779,100032540_1
1,0.488730,0.491613,0.490693,0.481814,0.492554,0.493867,0.497078,0.386821,0.499274,0.500586,...,0.651187,0.555331,0.557819,0.558858,0.555514,0.549948,0.129066,-0.008744,0.011832,100040721_1
2,0.567273,0.582619,0.571287,0.563135,0.584234,0.581544,0.571529,0.438586,0.569168,0.570418,...,0.629487,0.545610,0.546705,0.546267,0.557263,0.550167,0.252579,-0.307590,-0.001831,100040721_2
3,0.483056,0.489055,0.487805,0.485868,0.491894,0.495909,0.506374,0.389012,0.514075,0.518320,...,0.678532,0.586400,0.587484,0.588105,0.588859,0.584550,0.034011,-0.031420,0.148353,1002681492_1
4,0.445695,0.454399,0.454913,0.460774,0.458520,0.466002,0.486060,0.392390,0.500045,0.505943,...,0.672577,0.581859,0.582787,0.583007,0.590419,0.585309,0.080630,0.143988,0.273986,1004467229_1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3829,0.471343,0.453756,0.463752,0.434554,0.449902,0.448082,0.446691,0.340259,0.441700,0.436760,...,0.674656,0.545103,0.552199,0.557579,0.521551,0.515708,-0.094009,0.166907,-0.168874,image_train_0867
3830,0.453791,0.436309,0.448164,0.425786,0.434483,0.436197,0.444593,0.340202,0.446723,0.445582,...,0.664028,0.533001,0.539118,0.543963,0.520076,0.512023,-0.084151,0.242622,-0.078441,image_train_0868
3831,0.536831,0.554418,0.544344,0.539163,0.556562,0.555888,0.550812,0.419421,0.551688,0.553846,...,0.642112,0.528334,0.532459,0.535581,0.524266,0.520053,-0.146421,-0.234853,0.044059,image_train_0869
3832,0.534244,0.520356,0.525062,0.490569,0.515272,0.508841,0.493298,0.376543,0.480124,0.473075,...,0.688384,0.553510,0.561295,0.567499,0.522419,0.515140,-0.220850,-0.076772,-0.255706,image_train_0870


In [11]:
for col in ['pitch','yaw','roll']:
    IQR = df[col].quantile(0.75) - df[col].quantile(0.25)
    max = df[col].quantile(0.75) +3*IQR
    min = df[col].quantile(0.25) -3*IQR 
    outlierdf = df[(df[col] > max) | (df[col] < min)]
    df.drop(df[(df[col] > max) | (df[col] < min)].index, inplace=True)
outlierdf.shape

(22, 940)

In [12]:
df.iloc[:,0:468]= (df.iloc[:,0:468].values /df.iloc[:,1].values.reshape(-1,1))-1
df.iloc[:,468:936]= (df.iloc[:,468:936].values /df.iloc[:,469].values.reshape(-1,1))-1

In [13]:
X_train, X_test, Y_train, Y_test = train_test_split(df.iloc[:,0:-4],
df.iloc[:,-4:-1],test_size=0.2,random_state=0)
print(X_train.shape)
print(X_test.shape)

(6108, 936)
(1528, 936)


In [14]:
svmModel = make_pipeline(Normalizer(),
                    #  PCA(n_components=900),
                MultiOutputRegressor(SVR(C = 0.2,kernel="poly"),n_jobs=-1)
                )
svmModel.fit(X_train, Y_train)
predictions =  svmModel.predict(X_train.values)
print(mean_squared_error(Y_train, predictions))
print(r2_score(Y_train, predictions))
print("#######################################")
test_predictions =  svmModel.predict(X_test.values)
print(mean_squared_error(Y_test, test_predictions))
print(r2_score(Y_test, test_predictions))
print("#######################################")
# test_predictions =  svmModel.predict(X_test.values)
print(r2_score(Y_test['pitch'], test_predictions[:,0]))
print(r2_score(Y_test['yaw'], test_predictions[:,1]))
print(r2_score(Y_test['roll'], test_predictions[:,2]))

C:\Users\ahmed\AppData\Roaming\Python\Python310\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but Normalizer was fitted with feature names
  warnings.warn(


0.0028452971834005635
0.9071575616511819
#######################################


C:\Users\ahmed\AppData\Roaming\Python\Python310\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but Normalizer was fitted with feature names
  warnings.warn(


0.0030081959677620562
0.9012406326772148
#######################################
0.8081154706143057
0.9688644377842405
0.9267419896330984


In [7]:
import cv2 as cv
faces = mediapipe.solutions.face_mesh.FaceMesh()
cap = cv2.VideoCapture(0)
# frame = cv2.imread("../test/2177168298_1.jpg")
pit = yaw =rol = tdx = tdy =0
while(cap.isOpened()):
    # load the input image and convert it to grayscale
    _, frame = cap.read()

    results = faces.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    if results.multi_face_landmarks != None: 
        # looping over the faces in the frame
        for face in results.multi_face_landmarks:
            xarr=[]
            yarr=[]

            for landmark in face.landmark:
                print
                x = landmark.x
                y = landmark.y

                xarr.append(x)
                yarr.append(y)
                # note: the x and y values are scaled to the their width and height so we will get back their actual value in the frame
                shape = frame.shape 
                relative_x = int(x * shape[1])
                relative_y = int(y * shape[0])
                # cv2.putText(frame, str(relative_y), (int(relative_x),int(relative_y)), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0,255,0), 2)
                cv2.circle(frame, (relative_x, relative_y), radius=1, color=(0, 255, 0), thickness=2)
            arr = np.array([xarr,yarr]).reshape(1,-1)
            arr[:,0:468]= (arr[:,0:468] /arr[:,1].reshape(-1,1))-1
            arr[:,468:936]= (arr[:,468:936] /arr[:,469].reshape(-1,1))-1
            predict = svmModel.predict(arr)
        
            cv2.putText(frame,'yaw : '+str(predict[0,0]), (0, 20), cv2.FONT_HERSHEY_SIMPLEX,0.7, (246,255,12), 2)
            cv2.putText(frame,'pitch : ' + str(predict[0,1]), (0, 50), cv2.FONT_HERSHEY_SIMPLEX,0.7, (246,255,12), 2)
            cv2.putText(frame,'roll : '+ str(predict[0,2]), (0, 80), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (246,255,12),2)

            pit=predict[0,0]
            yaw=-predict[0,1]
            rol=predict[0,2]
            yawlook = ""
            pitlook = ""
            rolllook = ""
            if np.tan(yaw)>0.17632698070846498 and np.tan(yaw)<0.9999999999999999:
                yawlook ="look left"
            elif np.tan(yaw)<-0.17632698070846498 and np.tan(yaw)>-0.9999999999999999:
                yawlook =  "look right"

            #pitch look up and down
            if np.tan(pit)>0.17632698070846498 and np.tan(pit)<0.9999999999999999:
                pitlook = "look up"
            elif np.tan(pit)<-0.17632698070846498 and np.tan(pit)>-0.9999999999999999:
                pitlook = "look down"
            #roll shake left and right
            if np.tan(rol)>0.17632698070846498 and np.tan(rol)<0.9999999999999999:
                rolllook = "shake right"
            elif np.tan(rol)<-0.17632698070846498 and np.tan(rol)>-0.9999999999999999:
                rolllook = "shake left"
           
            cv2.putText(frame,'yaw : '+yawlook, (0, 220), cv2.FONT_HERSHEY_SIMPLEX,0.7, (246,255,12), 2)
            cv2.putText(frame,'pitch : ' + pitlook, (0, 250), cv2.FONT_HERSHEY_SIMPLEX,0.7, (246,255,12), 2)
            cv2.putText(frame,'roll : '+ rolllook, (0, 280), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (246,255,12),2)
            tdx=int(xarr[1] * shape[1])
            tdy=int(yarr[1] * shape[0])
            # bbox_height, bbox_width = frame.shape[:2]
                    
    cv2.imshow("Output",draw_axis(frame,pit,yaw,rol,tdx,tdy))

    k = cv2.waitKey(5) & 0xFF
    if k == 27:
        break
                    
cv2.destroyAllWindows()

C:\Users\ahmed\AppData\Roaming\Python\Python310\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but Normalizer was fitted with feature names
  warnings.warn(
C:\Users\ahmed\AppData\Roaming\Python\Python310\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but Normalizer was fitted with feature names
  warnings.warn(
C:\Users\ahmed\AppData\Roaming\Python\Python310\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but Normalizer was fitted with feature names
  warnings.warn(
C:\Users\ahmed\AppData\Roaming\Python\Python310\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but Normalizer was fitted with feature names
  warnings.warn(
C:\Users\ahmed\AppData\Roaming\Python\Python310\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but Normalizer was fitted with feature names
  warnings.warn(
C:\Users\ahmed\AppData\Ro

In [ ]:
import cv2 as cv
faces = mediapipe.solutions.face_mesh.FaceMesh()
cap = cv2.VideoCapture(0)
# frame = cv2.imread("../test/2177168298_1.jpg")
pit = yaw =rol = tdx = tdy =0
while(cap.isOpened()):
    # load the input image and convert it to grayscale
    _, frame = cap.read()

    results = faces.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    if results.multi_face_landmarks != None: 
        # looping over the faces in the frame
        for face in results.multi_face_landmarks:
            xarr=[]
            yarr=[]

            for landmark in face.landmark:
                print
                x = landmark.x
                y = landmark.y

                xarr.append(x)
                yarr.append(y)
                # note: the x and y values are scaled to the their width and height so we will get back their actual value in the frame
                shape = frame.shape 
                relative_x = int(x * shape[1])
                relative_y = int(y * shape[0])
                # cv2.putText(frame, str(relative_y), (int(relative_x),int(relative_y)), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0,255,0), 2)
                cv2.circle(frame, (relative_x, relative_y), radius=1, color=(0, 255, 0), thickness=2)
            predict = svmModel.predict(np.array([xarr,yarr]).reshape(1,-1))
        
            cv2.putText(frame,'yaw : '+str(predict[0,0]), (0, 20), cv2.FONT_HERSHEY_SIMPLEX,0.7, (246,255,12), 2)
            cv2.putText(frame,'pitch : ' + str(predict[0,1]), (0, 50), cv2.FONT_HERSHEY_SIMPLEX,0.7, (246,255,12), 2)
            cv2.putText(frame,'roll : '+ str(predict[0,2]), (0, 80), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (246,255,12),2)

            pit=predict[0,0]
            yaw=-predict[0,1]
            rol=predict[0,2]
            yawlook = ""
            pitlook = ""
            rolllook = ""
            if np.tan(yaw)>0.17632698070846498 and np.tan(yaw)<0.9999999999999999:
                yawlook ="look right"
            elif np.tan(yaw)<-0.17632698070846498 and np.tan(yaw)>-0.9999999999999999:
                yawlook =  "look left"

            #pitch look up and down
            if np.tan(pit)>0.17632698070846498 and np.tan(pit)<0.9999999999999999:
                pitlook = "look up"
            elif np.tan(pit)<-0.17632698070846498 and np.tan(pit)>-0.9999999999999999:
                pitlook = "look down"
            #roll shake left and right
            if np.tan(rol)>0.17632698070846498 and np.tan(rol)<0.9999999999999999:
                rolllook = "shake right"
            elif np.tan(rol)<-0.17632698070846498 and np.tan(rol)>-0.9999999999999999:
                rolllook = "shake left"
           
            cv2.putText(frame,'yaw : '+yawlook, (0, 220), cv2.FONT_HERSHEY_SIMPLEX,0.7, (246,255,12), 2)
            cv2.putText(frame,'pitch : ' + pitlook, (0, 250), cv2.FONT_HERSHEY_SIMPLEX,0.7, (246,255,12), 2)
            cv2.putText(frame,'roll : '+ rolllook, (0, 280), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (246,255,12),2)
            tdx=int(xarr[1] * shape[1])
            tdy=int(yarr[1] * shape[0])
            # bbox_height, bbox_width = frame.shape[:2]
                    
    cv2.imshow("Output",draw_axis(frame,pit,yaw,rol,tdx,tdy))

    k = cv2.waitKey(5) & 0xFF
    if k == 27:
        break
                    
cv2.destroyAllWindows()